In [1]:
import pandas as pd
import chromadb
import chromadb.config
from chromadb.config import Settings
import os

In [2]:
df=pd.read_csv("final.csv")

In [3]:
df

,Unnamed: 0,num,title,chunks,embeddings
0,0,9251120,maybe this time (2014),it could ve been just another summer but as ...,[-9.84467790e-02 -9.74959359e-02 5.34681901e-...
1,1,9251120,maybe this time (2014),a family like yours happy and complete it s...,[-1.18996575e-01 -4.86601964e-02 8.66751652e-...
2,2,9251120,maybe this time (2014),ve got to be kidding me steph the client ha...,[-6.74568191e-02 2.20672823e-02 3.27523984e-...
3,3,9251120,maybe this time (2014),t because i m giving this project to liana ...,[-6.42515197e-02 1.50788315e-02 1.63318217e-...
4,4,9251120,maybe this time (2014),your mufflers i mean your it s mama mae s ...,[-4.52403724e-02 -3.99174169e-02 1.20167047e-...
...,...,...,...,...,...
124481,124481,9417351,hot in cleveland s05 e09 bad george clooney (2...,basis but i am officially in the radio busin...,[-7.29084313e-02 -7.71497712e-02 5.78299053e-...
124482,124482,9460606,silk stalkings s04 e18 i know what scares you ...,api opensubtitles org is deprecated please im...,[-2.79885698e-02 -5.51499464e-02 -1.88457947e-...
124483,124483,9460606,silk stalkings s04 e18 i know what scares you ...,guys are good together don t give this thing...,[-7.58638307e-02 -5.97316539e-04 2.42358400e-...
124484,124484,9460606,silk stalkings s04 e18 i know what scares you ...,say uh what did you say it was it was j...,[-6.94301948e-02 -5.84800802e-02 4.34558839e-...


In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
df

,num,title,chunks,embeddings
0,9251120,maybe this time (2014),it could ve been just another summer but as ...,[-9.84467790e-02 -9.74959359e-02 5.34681901e-...
1,9251120,maybe this time (2014),a family like yours happy and complete it s...,[-1.18996575e-01 -4.86601964e-02 8.66751652e-...
2,9251120,maybe this time (2014),ve got to be kidding me steph the client ha...,[-6.74568191e-02 2.20672823e-02 3.27523984e-...
3,9251120,maybe this time (2014),t because i m giving this project to liana ...,[-6.42515197e-02 1.50788315e-02 1.63318217e-...
4,9251120,maybe this time (2014),your mufflers i mean your it s mama mae s ...,[-4.52403724e-02 -3.99174169e-02 1.20167047e-...
...,...,...,...,...
124481,9417351,hot in cleveland s05 e09 bad george clooney (2...,basis but i am officially in the radio busin...,[-7.29084313e-02 -7.71497712e-02 5.78299053e-...
124482,9460606,silk stalkings s04 e18 i know what scares you ...,api opensubtitles org is deprecated please im...,[-2.79885698e-02 -5.51499464e-02 -1.88457947e-...
124483,9460606,silk stalkings s04 e18 i know what scares you ...,guys are good together don t give this thing...,[-7.58638307e-02 -5.97316539e-04 2.42358400e-...
124484,9460606,silk stalkings s04 e18 i know what scares you ...,say uh what did you say it was it was j...,[-6.94301948e-02 -5.84800802e-02 4.34558839e-...


In [6]:
df['embeddings'] = df['embeddings'].apply(lambda x: (', '.join(x.strip('[] \n').split())).split(','))

In [7]:
df['embeddings'] = df['embeddings'].apply(lambda x: [float(val) for val in x])

In [8]:
embeddings = df['embeddings'].tolist()
ids = df['num'].astype(str).tolist()
documents = df['chunks'].tolist()

metadata = []
for index, row in df.drop(['chunks', 'embeddings'], axis=1).iterrows():
    metadata.append(row.to_dict())

folder_path = r"C:\Users\Acer\Desktop\Search Engine"

# Create the "engine" folder within the specified directory
engine_folder_path = os.path.join(folder_path, "engine")
os.makedirs(engine_folder_path, exist_ok=True)

# Create a PersistentClient object with the specified folder path
client = chromadb.PersistentClient(path=engine_folder_path)

# Now you can create a collection using this client
collection = client.create_collection(name="Search_Engine", metadata={"hnsw:space": "cosine"})

for doc, emb, doc_id, meta in zip(documents, embeddings, ids, metadata):
    collection.add(
        documents=doc,
        embeddings=emb,
        ids=doc_id,
        metadatas=meta
    )


Add of existing embedding ID: 9251120
Insert of existing embedding ID: 9251120
Add of existing embedding ID: 9251120
Insert of existing embedding ID: 9251120
Add of existing embedding ID: 9251120
Insert of existing embedding ID: 9251120
Add of existing embedding ID: 9251120
Insert of existing embedding ID: 9251120
Add of existing embedding ID: 9251120
Insert of existing embedding ID: 9251120
Add of existing embedding ID: 9251120
Insert of existing embedding ID: 9251120
Add of existing embedding ID: 9251120
Insert of existing embedding ID: 9251120
Add of existing embedding ID: 9211589
Insert of existing embedding ID: 9211589
Add of existing embedding ID: 9380845
Insert of existing embedding ID: 9380845
Add of existing embedding ID: 9380845
Insert of existing embedding ID: 9380845
Add of existing embedding ID: 9380845
Insert of existing embedding ID: 9380845
Add of existing embedding ID: 9301436
Insert of existing embedding ID: 9301436
Add of existing embedding ID: 9301436
Insert of exis

In [9]:
results = collection.query(
    query_texts=["up when he has fallen   is described by them as upsetting social order   to this inhumanity has come a man   whom god chose   and in that we believe   you ve overcome  i beg you to collect yourself   i speak of the messenger of god   muhammad teaches us to worship one god   to speak truth   to love our neighbors as ourselves   to give charity even a smile can be charity   to protect women from misuse   to shelter orphans   and to turn away from gods of wood and stone   i cannot keep still and hear this blasphemy   we are an ancient civilization   to call our gods wood and stone is to speak ignorantly of them   the idol the form is not what we worship   but the spirit that resides within the form   i agree that idolatry is not always fully understood   thank you   now let me bring him back to the women   god made woman to be the proper companion of man   she is different but equal   equal   we buy them   feed them  clothe them   use them  discard them   women equal to us   god created man from one male and one female   amr  you must respect in all woman the womb that bore you   why are your  guards so tongue tied   while this only guard is eloquent   god has spoken to us before   through abraham  noah  moses and through jesus christ   why should we be so surprised that god speaks to us now through muhammad   who taught you those names   they are named in the quran   i knew muhammad when he was an orphan minding sheep   and we knew christ as a carpenter   what christ says and what your muhammad says   is like two raised from the same land   they are lying to you they deny christ   you worship three gods  they say father  son and holy ghost  they say   what do you say of christ   they say god cannot have a son   christ is not the son of god   speak to me of christ   we say of christ what our prophet has taught us   that god cast his holy spirit into the womb of a virgin named  mary   and that she conceived christ  the apostle of god   the apostle he says not the son  not the son   what does your miracle  your quran say of the birth of our dear lord jesus christ   may i relate the words   come closer to me   in the name of god most gracious  most merciful   relate in the book  the story of mary   how she withdrew from her family to a place in the east   how we sent to her our angel  gabriel  who said   i am a messenger from your god   to announce the birth of a holy son to you   she said   how shall i  mary  have a son when no man has touched me   and gabriel replied   for your lord says  it will happen   we appoint him as a sign onto man   and a mercy from us"],
    n_results=5
)
results

{'ids': [['9362286', '9271799', '9513986', '9271787', '9462924']],
 'distances': [[0.4790276288986206,
   0.49517160654067993,
   0.5047545433044434,
   0.5152668952941895,
   0.521197497844696]],
 'metadatas': [[{'num': 9362286,
    'title': 'raised by wolves s02 e05 king (2022)'},
   {'num': 9271799, 'title': 'the bible s01 e07 mission (2013)'},
   {'num': 9513986, 'title': 'the popes exorcist (2023)'},
   {'num': 9271787, 'title': 'the bible s01 e06 hope (2013)'},
   {'num': 9462924, 'title': 'the monastery s01 e01 episode 1 1 (2022)'}]],
 'embeddings': None,
 'documents': [['woman singing the door that finally opens   with light flooding in   spilling out on the floor   the core that never was   now it will be   the bones of what was there before   every step  every beat   every thought  every breath  everything   is longing   pulling you from the sky   just like love will do   pulling you from the ground   just like love will do   support us and become vip member  to remove all ad